In [9]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont

In [3]:
def imwrite_unicode(path: str | Path, img: np.ndarray) -> bool:
    path = str(path)
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True


def imread_unicode(path: str | Path, flags=cv2.IMREAD_GRAYSCALE) -> np.ndarray | None:
    path = str(path)
    try:
        with open(path, "rb") as f:
            data = f.read()
        arr = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(arr, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e, "->", path)
        return None

In [11]:
def _fit_font_size(
    text: str,
    font_path: str,
    canvas_size: Tuple[int, int],
    margin: int,
    max_size: int,
    min_size: int,
) -> ImageFont.FreeTypeFont:
    W, H = canvas_size
    target_w = max(1, W - 2 * margin)
    target_h = max(1, H - 2 * margin)

    dummy_img = Image.new("L", (W, H), 0)
    dummy_draw = ImageDraw.Draw(dummy_img)

    best_font = ImageFont.truetype(font_path, size=min_size)
    lo, hi = min_size, max_size

    while lo <= hi:
        mid = (lo + hi) // 2
        font = ImageFont.truetype(font_path, size=mid)

        bbox = dummy_draw.textbbox((0, 0), text, font=font)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]

        if text_w <= target_w and text_h <= target_h:
            best_font = font
            lo = mid + 1
        else:
            hi = mid - 1

    return best_font


In [13]:
def _safe_filename_from_text(text: str, naming: str = "codepoint") -> str:
    if naming == "text":
        bad = r'<>:"/\|?*'
        cleaned = "".join("_" if c in bad else c for c in text).strip()
        if not cleaned:
            cleaned = "empty"
        return cleaned + ".png"

    cps = "-".join([f"U+{ord(ch):04X}" for ch in text])
    return f"{cps}.png"


In [15]:
def _render_single(
    text: str,
    font_path: str,
    image_size: Tuple[int, int],
    margin: int,
    max_font_size: int,
    min_font_size: int,
) -> Image.Image:
    W, H = image_size
    img = Image.new("L", (W, H), 0)  # black bg
    draw = ImageDraw.Draw(img)

    font = _fit_font_size(
        text=text,
        font_path=font_path,
        canvas_size=(W, H),
        margin=margin,
        max_size=max_font_size,
        min_size=min_font_size,
    )

    bbox = draw.textbbox((0, 0), text, font=font)
    text_w = bbox[2] - bbox[0]
    text_h = bbox[3] - bbox[1]

    x = (W - text_w) // 2 - bbox[0]
    y = (H - text_h) // 2 - bbox[1]

    draw.text((x, y), text, fill=255, font=font)  # white text
    return img


In [23]:
def render_sentence_as_chars(
    sentence: str,
    out_dir: str | Path,
    font_path: str,
    image_size: Tuple[int, int] = (256, 256),
    margin: int = 18,
    naming: str = "codepoint",
    overwrite: bool = True,
    max_font_size: int = 220,
    min_font_size: int = 30,
    skip_spaces: bool = True,
) -> Dict[str, List[Path]]:
    """
    sentence의 각 글자를 256x256 이미지로 렌더링하여 저장.

    출력 구조:
      out_dir/
        images/
          00_U+AC00.png ...

    return: {"images": [paths...]}
    """
    out_dir = Path(out_dir)
    images_dir = out_dir / "images"
    images_dir.mkdir(parents=True, exist_ok=True)

    chars: List[str] = []
    for ch in sentence:
        if skip_spaces and ch == " ":
            continue
        chars.append(ch)

    saved: List[Path] = []

    for i, ch in enumerate(chars):
        img_pil = _render_single(
            text=ch,
            font_path=font_path,
            image_size=image_size,
            margin=margin,
            max_font_size=max_font_size,
            min_font_size=min_font_size,
        )

        fn = f"{i:02d}_" + _safe_filename_from_text(ch, naming=naming)
        out_path = images_dir / fn

        if out_path.exists() and not overwrite:
            saved.append(out_path)
            continue

        arr = np.array(img_pil, dtype=np.uint8)  # 0~255, black bg
        ok = imwrite_unicode(out_path, arr)
        if not ok:
            raise RuntimeError(f"Failed to write: {out_path}")

        saved.append(out_path)

    return {"images": saved}


In [31]:
if __name__ == "__main__":
    sentence = "소프트웨어분석"
    font_path = r"D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\hangul_dataset\나눔손글씨 잘하고 있어.ttf"
    out_dir = r"D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\results\segment_results\printed_chars"

    render_sentence_as_chars(
        sentence=sentence,
        out_dir=out_dir,
        font_path=font_path,
        image_size=(256, 256),
        margin=18,
        naming="codepoint",
        overwrite=True,
        skip_spaces=True,
    )
    print("Done:", out_dir)


Done: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\results\segment_results\printed_chars
